In [3]:
using Flux, Zygote, Distributions, Plots, Optim

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/evanmunro/.julia/packages/CUDAdrv/aBgcd/src/CUDAdrv.jl:69


In [210]:
d=3
modeltrue = Chain(Dense(d, 2, tanh), Dense(2, 1))
true_loss(modeltrue, d, 10000)

[0.114816, 0.327052, 0.327464, 0.321805]


(Float32[0.126981, -0.821514, 0.124693, -0.547031, 0.124378, -1.80855, -0.107877, -0.603044, 2.73117, 0.00670278, 0.182362], 0.98504573f0)

In [197]:
steps = 1000
#updateParams!(Flux.params(model), [0.5, 0.5, 0.5])
#collect(Iterators.flatten([params(model)...]))
#a = 0.02
a = 0.02
model = Chain(Dense(d, 2, tanh), Dense(2, 1))
losses = zeros(steps)
for i in 1:steps
    losses[i] = experimentStep(model, frankelDGP, d, a)
    if i%10==0
        println(Flux.params(model))
    end
end
println(Flux.params(model))

Params([Float32[0.555762 -0.780258 0.6667; 0.147276 -0.0445889 -0.781528], Float32[0.00844722, 0.0647809], Float32[0.792513 -0.767169], Float32[-0.0895991]])
Params([Float32[0.622616 -0.724716 0.710297; 0.132134 -0.207742 -0.729869], Float32[0.0415751, 0.0052214], Float32[0.630695 -0.68328], Float32[-0.11491]])
Params([Float32[0.683449 -0.658942 0.704392; 0.122254 -0.320039 -0.698225], Float32[0.0356375, -0.0418952], Float32[0.528547 -0.696282], Float32[-0.118598]])
Params([Float32[0.740807 -0.62617 0.743013; 0.0834545 -0.40533 -0.608037], Float32[0.0788027, -0.0489198], Float32[0.470393 -0.718982], Float32[-0.124934]])
Params([Float32[0.676741 -0.585921 0.754452; -0.00884393 -0.452587 -0.591061], Float32[0.046604, -0.0282489], Float32[0.445945 -0.717493], Float32[-0.123794]])
Params([Float32[0.661142 -0.558458 0.737318; -0.0321023 -0.558678 -0.575079], Float32[0.0561033, -0.0141146], Float32[0.386928 -0.724768], Float32[-0.117286]])
Params([Float32[0.680181 -0.531433 0.698167; -0.0404

Params([Float32[0.747546 -0.158579 0.433029; -0.193792 -0.464725 -0.266423], Float32[-0.269258, -0.158204], Float32[0.420325 -0.866336], Float32[-0.110833]])
Params([Float32[0.733368 -0.187091 0.390548; -0.18365 -0.488717 -0.349622], Float32[-0.300052, -0.166151], Float32[0.50013 -0.841497], Float32[-0.114751]])
Params([Float32[0.756987 -0.204236 0.386286; -0.1628 -0.492529 -0.324716], Float32[-0.322099, -0.132303], Float32[0.494611 -0.877893], Float32[-0.108102]])
Params([Float32[0.715456 -0.158467 0.357068; -0.220273 -0.514883 -0.226904], Float32[-0.322701, -0.13812], Float32[0.476149 -0.880848], Float32[-0.0983976]])
Params([Float32[0.663836 -0.170317 0.379925; -0.238844 -0.514996 -0.224746], Float32[-0.310714, -0.181559], Float32[0.479978 -0.804942], Float32[-0.0930576]])
Params([Float32[0.686575 -0.186162 0.457744; -0.268351 -0.549887 -0.191744], Float32[-0.338217, -0.120506], Float32[0.488763 -0.798292], Float32[-0.0849744]])
Params([Float32[0.665895 -0.14377 0.465962; -0.228635 

In [209]:
lab = ["Experiment Loss", "Optimal Loss"]
plot(losses,xlabel="Step",ylabel="Loss",labels=lab)
plot!(zeros(length(losses)).+1.02, labels=lab)
savefig("nonconvex.pdf")

In [202]:
losses[1000]

1.029143214225769

In [175]:
function calc_loss(beta, model, y, z, gamma)
    x = zeros(size(z))
    mhat = deepcopy(model)
    updateParams!(Flux.params(mhat), beta)
    for i in 1:size(z, 2) 
        x[:, i] = frankelBR(mhat, z[:, i], gamma[i]) 
    end
    loss = mean((mhat(x).-y').^2)
    return loss 
end

function true_loss(model, d, n=1000)
    z = zeros(d, n)
    x = zeros(d, n)
    y = zeros(n)
    k = length(flatten(Flux.params(model)))
    gamma = zeros(n)
    
    for i in 1:n 
        y[i], x[:,i], z[:, i], gamma[i] = frankelDGP(model, d)
    end
    
    println(linReg(y,x'))
    res = Optim.optimize(beta -> calc_loss(beta, model, y, z, gamma), 
                                            zeros(Float32, k))
    bstar = Optim.minimizer(res)
    return bstar, Optim.minimum(res)
end

function perturb!(model, epsilon=0.03)
    policy = flatten(Flux.params(model))
    perturb = epsilon*rand([-1,1], size(policy))
    #don't need to perturb the intercept 
    perturb[length(policy)] = 0 
    policy .+= perturb 
    updateParams!(Flux.params(model), policy) 
    return perturb[1:(length(perturb)-1)]
end

function updateParams!(p::Params, flattened)
    i = 1 
    @assert length(flatten(p))==length(flattened)
    for b in p 
        for j in eachindex(b)
            b[j] = flattened[i] 
            i+=1 
        end
    end
end

function flatten(p::Params)
    return collect(Iterators.flatten([p...])) 
end

function frankelUtility(x, model, z, gamma)
    return -(model(x)[1] - sum(gamma.*(x.-z).^2))
end
    
function frankelBR(modeli, z, gamma) 
    res = Optim.optimize(x-> frankelUtility(x, modeli, z, gamma), z)
    xhat = Optim.minimizer(res)
    return xhat  
end
    
function frankelDGP(m, d)
    z_i = rand(Normal(),d)
    gamma = rand(Uniform(1,3),1)[1] 
    x_i = frankelBR(m, z_i, gamma)
    y_i =  mean(z_i) +rand(Normal(),1)[1] 
    return y_i, x_i, z_i, gamma  
    
end

function experimentStep(model, bestResponse,  
                            d=3, alpha=0.1, n=10000)
    k = length(flatten(Flux.params(model)))
    yhat = zeros(Float32, n) 
    perturbs = zeros(Float32, n, k-1)
    x = zeros(Float32, n, d)
    y = zeros(Float32, n) 
    for i in 1:n 
        modeli = deepcopy(model)
        perturbs[i,:] = perturb!(modeli)
        y[i], x[i, :], z, g = frankelDGP(modeli, d)
        yhat[i] = modeli(x[i, :])[1]
    end  
    
    losses = (yhat - y).^2  
    coef = linReg(losses, perturbs) 
    grads = coef[2:length(coef)]
    policy = flatten(Flux.params(model))
    policy[1:(length(policy)-1)] .-= alpha.*grads
    policy[length(policy)] += alpha*(mean(y) - mean(yhat))*2
    updateParams!(Flux.params(model), policy)
    return mean(losses)
end 

function linReg(y,x)
    x1 = [ones(length(y)) x ]#x.^2]
    m1 = inv(x1'*x1)*x1'*y 
    return m1
end 

linReg (generic function with 1 method)

In [ ]:
x, y = rand(3,5), rand(1,5)
loss(x, y) = Flux.mse(model(x),y)
gs = gradient(() -> loss(x,y), Flux.params(model))

In [ ]:
function true_loss(model, d=1, n=10000)
    z = zeros(d, n)
    y = zeros(n)
    gamma = zeros(n)
    
    for i in 1:n 
        y[i], a, z[:, i], gamma[i] = frankelDGP(model, d)
    end
    x = zeros(d, n)
    
    function calc_loss(beta)
        mhat = deepcopy(model)
        updateParams!(Flux.params(mhat), beta)
        for i in 1:n 
           x[:, i] = frankelBR(mhat, z[:, i], gamma[i]) 
        end
        loss = mean((mhat(x) .- y).^2)
        return loss 
    end
    res = Optim.optimize(utilityi, z)
    bstar = Optim.minimizer(res)
    return bstar, Optim.minimum(res)
end